In [1]:
import pandas as pd
import csv
import pyltr
import numpy as np
import datetime
import re
import time
import dawg
from nltk import ngrams
from heapq import nlargest
from operator import itemgetter


def get_single_ngram_features(query_input, ngram_dict):
    ngram_scores = []

    for n in range(1,7):
        sixgrams = ngrams(query_input.split(), n)
        ngram_score = 0
        for grams in sixgrams:
            try:
                ngram_score = ngram_score + float(ngram_dict[' '.join(grams)])
            except:
                pass
        ngram_scores.append(ngram_score)
    return ngram_scores

def get_ngram_features(inputs, ngram_dict):
    results = [[get_single_ngram_features(x, ngram_dict)] for x in inputs];
    return results


# Importing all the necessary dictionaries
suffixes = pd.read_csv("../data/Freq_background.csv", index_col='Unnamed: 0')
with open('../data/sorted_popular_queries.csv', mode='r') as infile:
    reader = csv.reader(infile)
    skipheader = next(reader)
    data=[tuple([line[1], int(line[2])]) for line in reader]
sortedpopulardict = dawg.IntCompletionDAWG(data)

with open('../data/ngram_dict.csv', mode='r') as infile:
    reader = csv.reader(infile)
    ngram_dict = {rows[0]:rows[1] for rows in reader}
# prefix_suffix_pairs_background = pd.read_csv("../data/prefix_suffix_pairs.txt")

data = []

top100k = suffixes.iloc[range(100000),:]
suffixes = []
top100ktuple = [tuple(x) for x in top100k.values]
# top100ktuple = [tuple([line["0"], int(line["counts"])])]
top100kdict = dawg.IntCompletionDAWG(top100ktuple)
top100k = []

/root/miniconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [27]:
# Read in training data
removed_session_training = pd.read_csv('../data/removed_session_training.csv', header=None, index_col=[0])
removed_session_training[1] = removed_session_training[1].str.replace('[\W_]+', ' ')

training_queries = removed_session_training[:1000].reset_index(drop=True)
candidate_prefixes = []
right_query = []

for j in range(len(training_queries)):
    current_query = str(training_queries[1][j])
    split_query = current_query.split(" ")
    suffix = ' '.join(split_query[1:])
    prefix = split_query[0] + " "

    for i in range(len(suffix)+1):
        candidate_prefixes.append(prefix + suffix[:i])
        right_query.append(current_query)
    
print("done")

done


In [28]:
# Run this to clear memory
all_candidates = []
relevant_candidate = []
qids =  []
ngram_features =  []

In [30]:
all_candidates = []
relevant_candidate = []
qids =  []
ngram_features =  []

time_taken_top100k = 0
split_time = 0 
appendtime = 0
ngramtime = 0
real_query_time = 0
total_time_begin = time.time()

for candidate_prefix_i in range(len(candidate_prefixes)):
    if candidate_prefix_i % 1000 == 0:
        print(str(candidate_prefix_i) + '/' + str(len(candidate_prefixes)))
    candidate_prefix = candidate_prefixes[candidate_prefix_i]
    original_query = right_query[candidate_prefix_i]
    
#     Getting the endterm for every prefix
    split_time_start = time.time()
    splitted = re.split('[\W_]+',candidate_prefix)
    if candidate_prefix[len(candidate_prefix)-1] == ' ':
        endterm = splitted[len(splitted)-2] + " "
    else:
        endterm = splitted[len(splitted)-1]
    split_time = split_time + time.time() - split_time_start
    
    # Creating the synthetic candidates
    start = time.time()
    top10suffix = nlargest(10, top100kdict.items(prefix=candidate_prefix), key=itemgetter(1))
    out = [i[0] for i in top10suffix]    
    preprefix = candidate_prefix[:-len(endterm)]
    outcombined1 = [preprefix + s for s in out]
    time_taken_top100k = time_taken_top100k + time.time() - start
    
    # Getting the top 50 real queries
    realquerystart = time.time()
    top50queries = nlargest(25, sortedpopulardict.items(prefix=candidate_prefix), key=itemgetter(1))
    keys_pop = [i[0] for i in top50queries]
    current_candidates = outcombined1
    current_candidates.extend(keys_pop)
    real_query_time = real_query_time + time.time() - realquerystart
    
    # Appending
    append_time_start = time.time()
    all_candidates.extend(current_candidates)    
    relevant_candidate.extend([s == original_query for s in current_candidates])
    qids.extend(np.ones(len(current_candidates)) * candidate_prefix_i)
    appendtime = appendtime + time.time() - append_time_start

    # Creating features
    ngram_time_start = time.time()
    ngram_features = np.append(ngram_features, get_ngram_features(current_candidates, ngram_dict))
    ngramtime = ngramtime + time.time() - ngram_time_start
    
total_time = time.time() - total_time_begin

print("Total time: " + str(total_time))
print("Top100k suffix time: " + str(time_taken_top100k))
print("Total split time: " + str(split_time))
print("Total append time: " + str(appendtime))
print("Total ngram time: " + str(ngramtime))
print("Total real query time: " + str(real_query_time))


0/11323
1000/11323
2000/11323
3000/11323
4000/11323
5000/11323
6000/11323
7000/11323
8000/11323
9000/11323
10000/11323
11000/11323
Total time: 63.08382844924927
Top100k suffix time: 0.44184350967407227
Total split time: 0.1052556037902832
Total append time: 0.19327688217163086
Total ngram time: 4.6560070514678955
Total real query time: 57.650792360305786


In [14]:
all_candidates

['www yahoo com',
 'www google com',
 'www myspace com',
 'www ebay com',
 'www google',
 'www ',
 'www hotmail com',
 'www mapquest com',
 'www myspace',
 'www msn com',
 'www yahoo com',
 'www google com',
 'www myspace com',
 'www ebay com',
 'www google',
 'www ',
 'www hotmail com',
 'www mapquest com',
 'www myspace',
 'www msn com',
 'www bankofamerica com',
 'www pogo com',
 'www irs gov',
 'www disneychannel com',
 'www disney channel com',
 'www ask com',
 'www mysapce com',
 'www yahoo',
 'www my',
 'www ebay',
 'www mapquest',
 'www wellsfargo com',
 'www nick com',
 'www my space com',
 'www walmart com',
 'www fafsa gov',
 'www askjeeves com',
 'www weather com',
 'www disney com',
 'www cartoonnetwork com',
 'www wachovia com',
 'www http',
 'www target com',
 'www google co',
 'www craigslist com',
 'www cingular com',
 'www google cm',
 'www myspace cm',
 'www macy s com',
 'www chase com',
 'www blackplanet com',
 'www aol com',
 'www amazon com',
 'www bankofamerica'

In [157]:
# query = input("Query: ")
# original_query = input("Original query: ")

# search_query = re.sub('[\W_]+',' ', query)
# original_query = re.sub('[\W_]+',' ', original_query)

# startt = time.time()
# splitted = re.split('[\W_]+',search_query)
# print(splitted)
# if query[len(query)-1] == ' ':
#     endterm = splitted[len(splitted)-2] + " "
# else:
#     endterm = splitted[len(splitted)-1]

# print("Endterm -> '" + endterm + "'")

# out = top100k[top100k['0'].str.startswith(endterm)].nlargest(10, 'counts')
# out2 = sortedpopular[sortedpopular['0'].str.startswith(search_query)].nlargest(40, 'counts')

# outcombined1 = query[:-len(endterm)] + out['0'].astype(str)
# time.time() - startt
# all_candidates = outcombined1.append(out2.iloc[:,0])
# relevant_candidate = (all_candidates == original_query).apply(float)

# train_input = pd.DataFrame({
#     'query': all_candidates,
#     'relevant': relevant_candidate,
#     'qid': 1
# })
  
# train_input['ngram_features'] = get_ngram_features(all_candidates, ngram_dict)
# print(train_input)


In [158]:
metric = pyltr.metrics.err.ERR(highest_score=1, gain_type='identity')

model = pyltr.models.LambdaMART(
    metric=metric,
    verbose=1,
)

TX2 = ngram_features.reshape(-1, 6) 
Ty2 = np.array(relevant_candidate)
Tqids2 = np.array(qids)
model.fit(TX2, Ty2, Tqids2)

 Iter  Train score    Remaining                           Monitor Output 
    1       0.1200        4.99m                                         
    2       0.1235        5.07m                                         
    3       0.1388        5.11m                                         
    4       0.1458        5.06m                                         
    5       0.1504        5.05m                                         
    6       0.1511        5.04m                                         
    7       0.1511        5.01m                                         
    8       0.1517        4.96m                                         
    9       0.1549        4.92m                                         
   10       0.1551        4.86m                                         
   15       0.1575        4.59m                                         
   20       0.1596        4.37m                                         
   25       0.1656        4.09m                   

In [ ]:
with open('../data/train.txt') as trainfile, \
        open('../data/vali.txt') as valifile, \
        open('../data/test.txt') as evalfile:
    TX, Ty, Tqids, _ = pyltr.data.letor.read_dataset(trainfile)
    VX, Vy, Vqids, _ = pyltr.data.letor.read_dataset(valifile)
    EX, Ey, Eqids, _ = pyltr.data.letor.read_dataset(evalfile)

In [161]:
EX = TX2 
Ey = Ty2
Eqids = Tqids2

Epred = model.predict(EX)

In [162]:
metric_MRR = pyltr.metrics.err.ERR(highest_score=1, gain_type='identity')

print('Random ranking:' + str(metric_MRR.calc_mean_random(Eqids, Ey)))
print('Our model:' + str(metric_MRR.calc_mean(Eqids, Ey, Epred)))

Random ranking:0.08816215706882063
Our model:0.18282111410843654


In [164]:
import MRR

In [165]:
sorted_pred_y = pyltr.util.sort.get_sorted_y(Ey, Epred)
evaluateMRR(Eqids, sorted_pred_y)

0: False
1: False
2: False
3: False
4: False
5: False
6: False
7: False
8: False
9: False


0.0

In [146]:
import pyltr.metrics.err
pyltr.metrics.err.ERR.evaluate = evaluate(self, qid, targets):
    residual = 1.0
    result = 0.0
    for i, t in enumerate(targets[:self.k]):
        assert t <= self.highest_score
        sprob = self._get_satisfied_prob(t)
        result += residual * sprob / (1.0 + i)
        residual *= (1.0 - sprob)
        if residual < _EPS:
            break
    return result

In [90]:
metric = pyltr.metrics.NDCG(gain_type='identity', k=10)
print('Our model:' + str(metric.evaluate_preds(Eqids, Ey, Epred)))

inputs = ['hello kitty', 'hello my name is jeff', 'hello i am martijn', 'hello who are you', 'hello goodbye', 'hello kitty', 'hello my name is jeff', 'hello i am martijn', 'hello who are you', 'hello goodbye']
EX = get_ngram_features(inputs,ngram_dict)
EX = np.append([], EX)
EX = EX.reshape(-1, 6) 
Ey = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1])
Eqids = [0,0,0,0,0,1,1,1,1,1]

Epred = model.predict(EX)
print(Epred)
print('Random ranking:' + str(metric.calc_mean_random(Eqids, Ey)))
print('Our model:' + str(metric.evaluate_preds(Eqids, Ey, Epred)))
output = pd.DataFrame({
    'inputs': inputs,
    'value': Epred
})
output.sort_values('value', ascending=False)

[ 0.87089953 -4.59820971 -3.75258981 -1.29213702 -0.10469242  0.87089953
 -4.59820971 -3.75258981 -1.29213702 -0.10469242]
Random ranking:0.5896918237758785


TypeError: unhashable type: 'list'

In [47]:
Eqids

[array([0., 0., 0., 0., 0.]), array([1., 1., 1., 1., 1.])]